# Pandas


Pandas (Python Data Analysis Library) is a swiss-army knife module that you'll find at the top of a huge proportion of notebooks. It is so popular that there's even an idiom for how to import it...

In [ ]:
import pandas as pd

To get a quick idea of the scope of pandas take a look at the autocomplete for pd.<TAB>

In [ ]:
pd.api

The fundamental objects in pandas are the Series and the DataFrame. Together they encapsulate how you will normally ingest, clean, manipulate and even visualize external data sources. Most of your work will use methods on these 
objects so we'll take a quick tour of the concepts they implement, and the idea of an Index.

## Series
The pandas Series object is basically a one dimensional indexed array. Unpacking that, it is something that will look like

| Index | Value |
|-------|-------|
|   0   |  0.12 |
|   1   |  0.24 |
|   2   |  0.36 |
|   3   |  0.48 |

One thing to notice is that the values are all of the same time (floating point here). The index is fairly flexible, it might be numbers (as here), strings, timestamps or something else. The main restriction is that they should be hashable. Let's create some series to play with

In [ ]:
# sf a series of floats from 1.0 to 5.0

# si a series of integers from 0 to 5

The `__repr__` includes the values we set along with the type of object we have stored (the values). As I mentioned, this is one of the attributes of a Series

In [ ]:
# Check `.dtype` of sf and si

So far, this looks a lot like a numpy array (or even just a list), but we can switch the indexing to suit our needs, by explicitly passing the `index=` arguement

In [ ]:
# sm a seris of 5 floats with the index being their values as words


Now the series is starting to look more like a dictionary, in fact, that's a pretty good way to construct series

In [ ]:
# sn a series created from a dictionary

When you look closely though, a Series has a few tricks that a dictionary doesn't...

In [ ]:
# Try slicing sn

N.B. label based indexes are _inclusive_ of the `stop` value. This is different from most other indexes you'll see in python. This can be a little confusion, but it basically boils down to the idea that there isn't always a natural "next" object in a hash


In [ ]:
# show the `.keys` of sn

# Check if 'four' is in the series index

Element by element statements evaluate to Booleans

In [ ]:
# Check where sn > 2


# Extract the values where sn > 2

There are also some indexing methods available to you `.loc`, `.iloc`, `.ix`. These might seem redundant, but actually they're useful in some contexts where ordinary indexing will bite you...

In [ ]:
sa = pd.Series(data=['apple', 'orange', 'pineapple'], index=[1, 3, 7])
sa

In [ ]:
sa[3]

In [ ]:
sa[:3]

When actually, you probably meant...

In [ ]:
sa.loc[:3]

`iloc` does the opposite and tells pandas you want to use the implicit style python notation

In [ ]:
sa.iloc[:2]

`ix` does something similar, but I've never been able to commit it to memory...😀

## DataFrames

Most of the time you will be using DataFrames rather than series, but dataframes can be thought of as collections of Series. They can actually be multidimensional, but for now think of them as a collection of `Series` columns

In [ ]:
d1 = pd.DataFrame({'floats': sm, 'ints': sn})
d1

# Check the index and colums of d1

When indexing a dataframe, the default is to give you the column

In [ ]:
# Display the 'ints' column of d1

# Check the type of the object you get back from this indexing

If you are looking for the row, then try `.loc` with the row index value

In [ ]:
# Get the `one` row of d1

You can also specify columns using the attribute notation...

In [ ]:
# Show the `.ints` column of d1

### Adding Columns
We can manipulate existing columns or create new ones, but simple transformations will return the transformed object rather than modifying in place. Many options have an `inplace=True` argument, but you can also just reassign the returned DataFrame to the same name.

When you slice or subset, pandas will usually try to get away with giving you a view on the existing dataframe rather than giving you a new object. This is cheaper in terms of memory allocated and also performance penalty of the copies, sometimes 

In [ ]:
# Double the d1['ints'] column

# Check that d1 is unchanged

# divide the ints column of d1 by twice the floats column

### DataFrame Attributes

We've already seen some of the attributes of the DataFrame (column etc.) but there are quite a few available, take a look at `dtypes`, `ndim`, `shape`

### DataFrame Methods

There are *lots* of methods for operating on DataFrames, have a look at the tab completion and explore the documentation for them. In particular, take a look at the help for `describe`, `head` and `tail`. These are great for orienting yourself with a new dataset

In [ ]:
# Describe the d1 DataFrame

## Working with External Data

Data comes in many, many forms from simple csv/json files, real-time APIs, structured binary files and many others. Try pd.read_<TAB> to see some of the available options. `read_csv` and `read_json` are the workhorses. We'll start with pd.read_csv which is more fexible but many of the arguments to read_csv will have equivalents for the other functions. `read_json` can be useful for quicky interacting with public APIs which will commonly publish JSON endpoints.
    
    
We need a CSV to work with. The city of Vancouver has an [open data catalog](https://vancouver.ca/your-government/open-data-catalogue.aspx), which has CSV for some of the datasets. You can download the file directly if you want, e.g.
    
!curl -O ftp://webftp.vancouver.ca/OpenData/csv/CommunityGardensandFoodTrees.csv

but `read_csv` will also happily read from remote for you

In [ ]:
gardenDF = pd.read_csv(
    "ftp://webftp.vancouver.ca/OpenData/csv/CommunityGardensandFoodTrees.csv",
    encoding='latin1'
)
gardenDF.shape

So there are 168 rows, with 19 columns, here are the fist few rows

So we know when the gardens were created (`YEAR_CREATED`), and where the are (`LATITUDE`, `LONGITUDE`).

Now it is time to clean the data. This is a hugely important step and in research will eat a lot of your time, but it can be worth it. First let's look at the index, the default is to index by integer, but we could have picked any column instead. It looks like the first column is unique (`MAPID`) and so let's use that (chosing the index right can make your life much easier when adding data or combining multiple DataFrames). The `inplace=True` argument means modify the existing dataframe rather than returning a modified copy

In [ ]:
# `.set_index` to mapID in place

# Use the `.unique` method on the `YEAR_CREATED` column

`Pre-2010`, `pre-1970` and `pre 200` are kind of usless (and inconsistent!) so let's toss them

In [ ]:
for badLabel in ['Pre-2010', 'pre-1970', 'pre 2000']:
    gardenDF = gardenDF[gardenDF['YEAR_CREATED'] != badLabel]

# Check the unique values in `YEAR_CREATED` again

There's still a `nan`, and the years are strings (numbers would be better or even dates). Pandas is pretty smart about dealing with missing data, but when it gets in the way there are functions like dropna() which will tell pandas to remove them. If we try to convert the `YEAR_CREATED` column to an integer blindly, it will barf on the `nan`.

In [ ]:
# Use the `.dropna` and `astype` methods to make everything in YEAR_CREATED an integer


notice that we still have a `NaN` (formally, `NaN` is a float so our column was "promoted"). One of the Big advantages of Pandas is that it will do something sensible with missing data rather than just barfing.

In [ ]:
gardenDF['YEAR_CREATED'].describe()

Let's get a bit more visual, again notice that the `NaN` is just safely ignored.

In [ ]:
%matplotlib inline

# Create a histogram of the YEAR_CREATED

Many of pandas ingenstion methods will let you do the transformations when you first read in the data by passing arguments to `read_csv` (or `read_json`, ...). Some of the more useful options are

  * **delimiter=**: Sometimes a csv is a tsv, tabs are evil
  * **names=**: Pass a list of names to use for the columns
  * **usecols=**: Only slurp up a subset of columns
  * **skiprows=**: Ignore a number of rows at the top of the file
  * **na_values=**: Flag values which the CSV author used to indicate missing data, e.g. -1
  * **encoding=**: ...
  * **converters=**: Do some transformation on the columns before importing them
  * **parse_dates**=: Turn strings into dates
  
For the last two options you normally have to add a bit of logic to help pandas. For dates this might be because you  columns for years, months and days and you need to combine them, or you have to look for a timezone or something. This seems like a hassle, but it is usually worth it. Once you have a column (or an index) as a datetime object you can index it very flexibly (e.g. you can ask for all the rows which fall on a weekend in the PTD timezone)

In [ ]:
gardenDF = pd.read_csv(
    "ftp://webftp.vancouver.ca/OpenData/csv/CommunityGardensandFoodTrees.csv",
    encoding='latin1',
    na_values={'YEAR_CREATED': ['Pre-2010', 'pre-1970', 'pre 2000', 'nan']},
    index_col='MAPID',
    dtype={'YEAR_CREATED': float}
)
gardenDF['YEAR'] = pd.to_datetime(gardenDF['YEAR_CREATED'], format='%Y.0')
gardenDF[['YEAR_CREATED', 'YEAR']].head()

In [ ]:
# Check the `NAME` of the gardens created before 1950

## TimeSeries

We've already talked about time and date handling a bit, but I use this a lot, so we can talk about it more! Pandas was created to handle Financial data and do financial modeling. This lineage has given pandas really excellent time handling. The main objects are

  * **Time Stamps**: Specific points in time usually recorded to the second or nanosecond
  * **Time Intervals/Time Deltas**: These types lets you do arithmetic on time objects

We need some dates to play with. There's a convenience function called `to_datetime` which can convert many "human readable" dates to a pd.Timestamp object

In [ ]:
moonwalk = pd.to_datetime('July 20, 1969, 20:17 UTC')
moonwalk

Timestamps have attributes which let you extract days, year, etc. Normally these will be reported as numbers, but the strftime method supports the usual format specifiers (The correspond with the libc specifiers, here's a reference http://strftime.org/)

In [ ]:
print(f"The moon walk took place on a {moonwalk.strftime('%A')}")

In [ ]:
# Use pd.datetime.now() to find out how long ago the Apollo 11 moon walk was
# see also `from dateutil import relativedelta`

Let's look at another sample dataset, it contains three colums

  * year
  * month
  * passengers
  
We can combine the year and month to create a date, then we can use the result as the index for a single column dataframe

In [ ]:
flightsDF = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv',
    parse_dates=[['year','month']],
    index_col='year_month'
)
flightsDF.head()

In [ ]:
# Check the dtypes of the DataFrame

Now we can index based on date. Let's look at the number of flights in the 1951

In [ ]:
# Select the flights between 1951 and 1952 and plot as a bar graph

Timestamp objects can also deal with arithmetic

In [ ]:
# Compare the last and first elements of the index to find out the time range of the data

One extremely useful feature with time series is the ability to resample existing time series. For example, we could resample the flight data into year long bins and look at how the mean passenger count increased

In [ ]:
# Resample flightsDF as yearly data, find the mean number of passengers and do another bar plot

## Grouping, Joining, Concatenating

You can get pretty far by jamming everything into a single dataframe, but sometimes you might want to do aggregate operations within a dataframe (e.g. group together all of the rows by year and show the mean value of some other column). Alternatively you might want to add new rows to or columns to an existing DataFrame or join dataframes based on shared key.


In [ ]:
# Read the following csv into a dataframe called carsDF
#  https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv


# Calculate the mean value of the mpg column

### Groupby
Let's group things by number_of_cylinders and see how that affects mpg...

In [ ]:
# groupby then find the mean

Calling groupby on it's own will give you a `DataFrameGroupBy` object, you have to tell it what you want to do with the groups to actually see some results, this can be convenient if you want to look at different aggregate functions on the same groups.

In [ ]:
carsDFbyCylinders = carsDF.groupby('cylinders')
carsDFbyCylinders

The typical aggregate operations are things like

  * mean()
  * sum()
  * median()
  * min()/max()

Group the carsDF by model_year and look at the median mpg (don't include the other columns)

You can actually do much more with groupby, you can iterate over the groups

In [ ]:
# Iterate over the carsDFbyCylinders and spit out the number of cars in each group


How are there possibly 4 cars with 3 cylinders?!

We can apply multiple operations at the same time. The `.aggregate()` method can take a list of the operations you want to perform (e.g. ["max", "min"])

We can filter based on group, this is a silly example, but group the cars by cylinder, then show me all of the groups with a mean mpg > 15.

In [ ]:
carsDFbyCylinders.filter(lambda x: x['mpg'].mean()>15).head()

The transform method lets you perform a group operation then use the results to update the rows. For example, we could calculate mean values for our groups, then look at how individual cars perform relative to that

In [ ]:
carsDFbyCylinders.transform(lambda x: x - x.mean()).head()

There is also an apply method which is even more general and will let you apply an arbitrary function to the group results.

## Concatenate & Join

There are a handful of functions which handle concatenation. The main workhorse is `pd.concat`, but there are some convenience functions which will let you avoid passing lots of arguments to concat. You can combine `Series` and `DataFrames` but we'll jump straight to `DataFrames`,

In [ ]:
s1 = pd.DataFrame(
    {
        'upper': ['A', 'B', 'C'], 
        'lower': ['a', 'b', 'c']
    }, 
    columns=['upper', 'lower'], 
    index=[1,2,3]
)

s2 = pd.DataFrame(
    {
        'upper': ['D', 'E', 'F'],
        'lower': ['d', 'e', 'f']
    }, 
    columns=['upper', 'lower'],
    index=[4,5,6]
)

pd.concat([s1,s2])

Notice that we enclosed the thigs we want to join as some sort of iterable (a `list` here). What happens if one of the columns is missing from the data frame?

What if we wanted to add columns rather than rows

In [ ]:
t1 = pd.DataFrame({'upper': ['A', 'B', 'C'], 'lower': ['a', 'b', 'c']}, columns=['upper', 'lower'], index=[1,2,3])
t2 = pd.DataFrame({'greek': ['α', 'β', 'γ']}, index=[1,2,3])

pd.concat([t1, t2], axis='columns')

`pd.concat` will accept duplicate indices, but normally that indicates a problem with the data normalization. `concat` has a `verify_index` argument which can check for these problems and you can specify what you want to do with duplicates manually.

concat will often result in `NaN`s because some columns might not exist in both/all frames.

In [ ]:
d4 = pd.DataFrame({'fruit': ['apple', 'orange'], 'veg': ['brocolli', 'carrot'], 'tree': ['cedar', 'alder']})
d5 = pd.DataFrame({'veg': ['onion', 'potato'], 'fruit': ['banana','grape']})
pd.concat([d4,d5], sort=False, ignore_index=True)

Since the index wasn't important here, I threw it away and just accepted a new one.

In the general case, joining DataFrames can get complex. The concat method can take a `join` keyword to specify a database like join stragegy (inner or outer), but `pd.merge` is a bit more flexible. It implements the usual relations

  * one-to-one (similar to a concat)
  * many-to-one
  * many-to-many
  
For the many-to-one case here is an example

In [ ]:
adf1=pd.DataFrame({
    'class': ['insect', 'spider'], 
    'legs': [6, 8]}
)

adf2=pd.DataFrame({
    'name': ['molly', 'anna', 'stephen', 'mica'], 
    'class': ['insect','insect','spider','insect']}
)                     

print(adf1); print(adf2)

In [ ]:
pd.merge(adf1, adf2)

The _many-to-one_ is many different rows in adf2 being mapped to a single row in adf1 (insects). `pd.merge` also accepts a selection of keyword arguments so you can manually specify which columns to join, patch up name differences etc.